In [33]:
import import_ipynb
import configparser
import numpy as np
#
import buildDescriptors as bd


# MachineLeranedMC

In [37]:
class MachineLeranedMC( bd.ParseConfiguration,
                        bd.EnergyBarrier,
                      ):
    '''
    Performs Machine Learned Monte Carlo Swaps
    '''
    
    def __init__(self,
                 confParser, 
                 verbose = False
                ):
        
#         bd.ParseConfiguration.__init__(self, confParser, verbose = verbose )
        self.verbose     =  verbose 
        self.confParser  =  confParser
        
    def Parse(self,fp):
        self.lmpData = lp.ReadDumpFile( '%s'%(fp) ) 
        self.lmpData.GetCords( ncount = sys.maxsize)
        if self.verbose:
            print('elapsed time=%s s'%(time.time()-t0))
            print('time steps:',self.lmpData.coord_atoms_broken.keys())
            display(self.lmpData.coord_atoms_broken[0].head())

            
            self.perAtomData = np.loadtxt(fp)
        
    def GetDescriptors( self ):
        '''
        Compute structural descriptors 
        '''
        
        bd.EnergyBarrier.__init__( self,
                                  None,#'%s/EVENTS_DIR'%self.confParser['input files']['input_path'],
                                  None,#'%s/EVLIST_DIR'%self.confParser['input files']['input_path'],
                                  self.lmpData,
                                  None,# self.lmpDisp,
                                   verbose    = self.verbose,
                                   nconf      = 2, #--- only two events
                                   confParser = self.confParser,
                                   species    = ['Ni'], #'Ni Co Cr'.split()
                                   r_cut      = eval(self.confParser['EnergyBarrier']['r_cut']),
                                   dr         = eval(self.confParser['EnergyBarrier']['dr']),
                                   scale      = eval(self.confParser['EnergyBarrier']['scale']),
                                   n_max      = 8,
                                   l_max      = 6,

                      )
        
#        self.perAtomData = self.lmpData
        self.SetDescriptors(
                      #soap = False,
                      #acsf = True,   
                      gr = True,
                     )
        
    def GetDefects( self ):
        '''
        Classify Defects
        '''
        
        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")

        #---------------
        #--- zscore X
        #---------------        
        X      = np.c_[self.descriptors ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        #--- predict classes
        predict_x = model.predict( X ) 
        self.predict_classes = np.argmax( predict_x, axis=1 )
    
    
    def GetDisp( self ):
        '''
        Predict Displacements
        '''

        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")
        
        #---------------
        #--- zscore X
        #---------------        
        filtr  = self.predict_classes == 1
        X      = np.c_[self.descriptors[ filtr ] ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        self.predict_disp = model.predict( X )
        
    
    def BuildCatalog( self ):
#         rate_constant_prefactor = ?
        columns = 'AtomId barrier true_rate dx dy dz'.split()


## main()

In [38]:
#--- parse conf. file
confParser = configparser.ConfigParser()
confParser.read('configuration.ini')
print('conf. file sections:',confParser.sections())



mlmc  =  MachineLeranedMC(confParser,verbose = True)
    
#--- parse atom positions
mlmc.Parse('%s/%s'%(confParser['ml mc']['input_path'],confParser['ml mc']['dump_file']))
    
#--- total displacements
# !mkdir disp
# mlmc.Displacement( 'dumpFile/dump.xyz',
#                     'disp/disp.xyz'
#                  )

#--- build descriptors
mlmc.GetDescriptors()

#--- identify defects
mlmc.GetDefects()

#--- predict diffusion paths
mlmc.GetDisp()


#--- build catalog
mlmc.BuildCatalog()

conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net', 'ml mc']


AttributeError: 'NoneType' object has no attribute 'coord_atoms_broken'

In [24]:
mlmc.descriptors

array([[1.36621930e-19, 1.60528039e-17, 2.82456009e-15, ...,
        1.75714630e-12, 1.49995887e-14, 1.90623576e-16],
       [1.35967458e-19, 1.60355702e-17, 2.82406503e-15, ...,
        1.75490775e-12, 1.49717095e-14, 1.90609413e-16],
       [1.34396746e-19, 1.58598701e-17, 2.79114211e-15, ...,
        1.75994958e-12, 1.50351142e-14, 1.90213878e-16],
       ...,
       [1.37384158e-19, 1.61441969e-17, 2.84431702e-15, ...,
        1.77214538e-12, 1.51161652e-14, 1.92545299e-16],
       [1.35808914e-19, 1.59907648e-17, 2.81922784e-15, ...,
        1.75088518e-12, 1.49237865e-14, 1.90663296e-16],
       [1.36070544e-19, 1.59994777e-17, 2.82062132e-15, ...,
        1.75260464e-12, 1.49392122e-14, 1.90906469e-16]])

In [25]:
dir(mlmc)

['DefectFreeCrystal',
 'Displacement',
 'GetCenterAtom',
 'GetDescriptors',
 'GetEventid',
 'LabelDefects',
 'LabelDefectsUpdated',
 'Parse',
 'ParseDiffusionPath',
 'ParseEvList_dir',
 'ParseEvents_dir',
 'Print',
 'PrintDescriptors',
 'PrintPerAtomData',
 'ReadConfigurationFile',
 'SetDescriptors',
 'SplitByType',
 'WignerSeitz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'box',
 'confParser',
 'datFile',
 'descriptors',
 'dr',
 'events_dir',
 'evlist_dir',
 'l_max',
 'lib_path',
 'lmpData',
 'lmpDisp',
 'n_max',
 'nconf',
 'perAtomData',
 'positions',
 'r_cut',
 'scale',
 'shape',
 'species',
 'verbose']